In [ ]:
from IPython.display import clear_output

import datetime
import time
import os


from tools import (
    play_beep,
    print_account_summary,
    print_order,
    print_all_openorders,
    get_all_openorders,
)

from IPython.display import display
from IPython.core.display import HTML

play_beep()

display(HTML("<style>.output_subarea { overflow: auto; }</style>"))

In [ ]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=3124)
util.logToConsole()

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

In [ ]:
ib.cancelMktDepth(contract)

In [ ]:
print(ib.positions(account="U10394496"))

In [24]:
# default reset values
open_trade = None
close_trade = None
# get a specific order by id

# trades = get_all_openorders(ib)
# open_trade = [trade for trade in trades if trade.order.permId == 449873795]
# print_order(open_trade)

future = [
    pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
][0]

tick_tp = 10
ticrement = 0.25

print(future)
print()
print_account_summary(ib=ib)
print()
print_all_openorders(ib=ib)

Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=-2.0, avgCost=350391.61917955)

AccountType : INDIVIDUAL
Cushion : 0.77545
LookAheadNextChange : 1714570200
AccruedCash : 0.00
AvailableFunds : 164916.98
BuyingPower : 677788.96
EquityWithLoanValue : 169447.24
ExcessLiquidity : 169447.24
FullAvailableFunds : 164916.98
FullExcessLiquidity : 169447.24
FullInitMarginReq : 49832.90
FullMaintMarginReq : 45302.63
GrossPositionValue : 3764.95
InitMarginReq : 49832.90
LookAheadAvailableFunds : 179866.85
LookAheadExcessLiquidity : 183038.03
LookAheadInitMarginReq : 34883.03
LookAheadMaintMarginReq : 31711.84
MaintMarginReq : 45302.63
NetLiquidation : 218514.82
PreviousDayEquityWithLoanValue : 0.00
RegTEquity : 0.00
RegTMargin : 0.00
SMA : 623475.35
TotalCashValue : 214749.87

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	449873795	Submi

In [47]:
# t1 = get_all_openorders(ib)[1]
# print(t1)

open_trade = None
close_trade = None

In [50]:
## SELL TO OPEN SCALP 0.1
########################################

sell_15_10_scalp = {
    "strategy": "SELL TO OPEN SCALP 15/10",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "ask",
    "open_ticks": 10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "BUY",
    "close_ref": "open_price_fill",
    "close_ticks": -10,
}

buy_10_10_scalp = {
    "strategy": "BUY TO OPEN SCALP 10/10",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "bid",
    "open_ticks": -10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "SELL",
    "close_ref": "open_price_fill",
    "close_ticks": 10,
}

strategy_details = buy_10_10_scalp

print(strategy_details)

while True:
    clear_output(wait=True)  # Clear the output before printing new output

    # first order of the strategy
    if open_trade is None and close_trade is None:
        action = strategy_details["open_action"]
        qty = strategy_details["open_qty"]

        if strategy_details["open_ref"] == "bid":
            price_ref = ticker.domBids[0].price
        elif strategy_details["open_ref"] == "ask":
            price_ref = ticker.domAsks[0].price
        elif strategy_details["open_ref"] == "mid":
            price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
        elif strategy_details["open_ref"] == "last":
            raise Exception("Not implemented")

        lmtPrice = (
            price_ref
            + strategy_details["open_ticks"] * strategy_details["tick_increment"]
        )
        print(
            f"Placing open trade: {action}, {strategy_details['open_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n"
        )
        print()

        if strategy_details["open_type"] == "LIMIT":
            open_order = LimitOrder(
                action=action,
                totalQuantity=qty,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
        else:
            raise Exception("Not implemented")

        open_trade = ib.placeOrder(contract, open_order)
        open_order_ts = datetime.datetime.now()

    print("OPEN ORDER::")
    print_order(open_trade)
    print()

    if open_trade is not None:
        if open_trade.orderStatus.status == "Submitted" and close_trade is None:
            print(
                f"Waiting to get filled on order #{open_trade.order.permId} ({open_trade.orderStatus.status})\n"
            )

            if datetime.datetime.now() - open_order_ts > datetime.timedelta(minutes=1):
                print("Cancelling order due to timeout:")
                ib.cancelOrder(open_trade.order)
                print()

        if open_trade.orderStatus.status == "Filled" and close_trade is None:
            action = strategy_details["close_action"]
            qty = strategy_details["close_qty"]

            if strategy_details["close_ref"] == "open_price_fill":
                price_ref = open_trade.orderStatus.avgFillPrice
            if strategy_details["close_ref"] == "bid":
                price_ref = ticker.domBids[0].price
            elif strategy_details["close_ref"] == "ask":
                price_ref = ticker.domAsks[0].price
            elif strategy_details["close_ref"] == "mid":
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
            elif strategy_details["close_ref"] == "last":
                raise Exception("Not implemented")

            lmtPrice = (
                price_ref
                + strategy_details["close_ticks"] * strategy_details["tick_increment"]
            )
            print(
                f"Placing close trade: {action}, {strategy_details['close_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n"
            )

            if strategy_details["close_type"] == "LIMIT":
                close_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception("Not implemented")

            close_trade = ib.placeOrder(contract, close_order)
            ib.sleep(1)
            play_beep()

        elif (
            open_trade.orderStatus.status == "Inactive"
            or open_trade.orderStatus.status == "Cancelled"
        ) and close_trade is None:
            print("***** order is inactive *****")
            print(open_trade.log)
            print("*****************************")
            open_trade = None

    print(f"CLOSE ORDER::")
    print_order(close_trade)
    print()

    if close_trade is not None:
        if close_trade.orderStatus.status == "Filled":
            play_beep()
            print(
                "Close trade filled @ {}\n".format(close_trade.orderStatus.avgFillPrice)
            )
            open_trade = None
            close_trade = None

            # small break before next trade
            for i in range(5):
                print(".", end="")
                time.sleep(1)

    print("ALL OPEN ORDERS::")
    print_all_openorders(ib=ib)
    print()

    # get current position of contract
    future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"][0]
    print(f"NQM2024: {ib.ticker(contract=contract).domBids[0].price}")
    print(
        f"Position: {future.contract.symbol} {future.position} @ {future.avgCost/float(contract.multiplier)}"
    )
    # acct_fields = ib.accountSummary(account="U10394496")
    print(
        f"NetLiquidation: {[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"RealizedPnL: {[v for v in ib.accountValues() if v.tag == 'RealizedPnL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"UnrealizedPnL: {[v for v in ib.accountValues() if v.tag == 'UnrealizedPnL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"OptionMarketValue: {[v for v in ib.accountValues() if v.tag == 'OptionMarketValue' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"TotalCashBalance: {[v for v in ib.accountValues() if v.tag == 'TotalCashBalance' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"FuturesPNL: {[v for v in ib.accountValues() if v.tag == 'FuturesPNL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )

    for i in range(3):
        print(".", end="")
        time.sleep(1)

2024-05-01 07:29:02,508 ib_async.ib INFO placeOrder: New order Trade(contract=Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=LimitOrder(orderId=287, clientId=3124, action='BUY', totalQuantity=1, lmtPrice=17468.75, account='U10394496'), orderStatus=OrderStatus(orderId=287, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 5, 1, 12, 29, 2, 507766, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')


Placing open trade: BUY, LIMIT, totalQuantity 1, lmtPrice 17468.75


OPEN ORDER::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	0	PendingSubmit	BUY	0.0	0.0		17468.75	

CLOSE ORDER::
None

ALL OPEN ORDERS::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	449873795	Submitted	BUY	0.0	4.0		15705.0	
NQ	1939819158	Submitted	BUY	0.0	1.0		17420.25	
NQ	1939819171	PendingSubmit	BUY	0.0	1.0		17468.75	
NQ	1939819075	Submitted	SELL	0.0	1.0		17519.5	
NQ	253828840	Submitted	SELL	0.0	1.0		17742.0	

NQM2024: 17471.25
Position: NQ -1.0 @ 17458.3846188425
NetLiquidation: 220313.5232
RealizedPnL: 2715.91
UnrealizedPnL: 854.85
OptionMarketValue: 4195.51
TotalCashBalance: 216118.0138
FuturesPNL: 847.34
.

In [ ]:
ib.accountValues()

In [ ]:
future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"][0]
future.position

In [ ]:
ib.ticker(contract=contract).domBids[0].price

In [ ]:
print(close_trade)
close_trade = ib.placeOrder(contract, close_order)
close_trade.log

In [ ]:
print_all_openorders(ib=ib)

In [ ]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != "NQ":
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )

In [30]:
trades_df = util.df(ib.trades())
trades_df

,contract,order,orderStatus,fills,log,advancedError
0,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=253828897, action='SELL', totalQu...","OrderStatus(orderId=0, status='Cancelled', fil...",[],[],
1,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=1939817375, action='BUY', totalQu...","OrderStatus(orderId=0, status='Cancelled', fil...",[],[],
2,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=253828896, action='BUY', orderTyp...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 4,...",
3,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=1939817374, action='BUY', totalQu...","OrderStatus(orderId=0, status='Cancelled', fil...",[],[],
4,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=253828899, action='SELL', totalQu...","OrderStatus(orderId=0, status='Cancelled', fil...",[],[],
...,...,...,...,...,...,...
535,"Contract(secType='FUT', conId=620730920, symbo...","LimitOrder(orderId=193, clientId=3124, permId=...","OrderStatus(orderId=193, status='Cancelled', f...",[],"[TradeLogEntry(time=datetime.datetime(2024, 5,...",
536,"Contract(secType='FUT', conId=620730920, symbo...","LimitOrder(orderId=194, clientId=3124, permId=...","OrderStatus(orderId=194, status='Cancelled', f...",[],"[TradeLogEntry(time=datetime.datetime(2024, 5,...",
537,"Contract(secType='FUT', conId=620730920, symbo...","LimitOrder(orderId=195, clientId=3124, permId=...","OrderStatus(orderId=195, status='Cancelled', f...",[],"[TradeLogEntry(time=datetime.datetime(2024, 5,...",
538,"Contract(secType='FUT', conId=620730920, symbo...","LimitOrder(orderId=196, clientId=3124, permId=...","OrderStatus(orderId=196, status='Filled', fill...","[(Contract(secType='FUT', conId=620730920, sym...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",


In [ ]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect("trades.db")

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create the "trades" table if it doesn't exist
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS trades (
        contract TEXT,
        order TEXT,
        orderStatus TEXT,
        fills TEXT,
        log TEXT,
        advancedError TEXT
    )
"""
)

# Convert the DataFrame to a list of tuples
trades_data = trades_df.to_records(index=False).tolist()

# Insert the data into the "trades" table
cursor.executemany(
    """
    INSERT INTO trades (contract, order, orderStatus, fills, log, advancedError)
    VALUES (?, ?, ?, ?, ?, ?)
""",
    trades_data,
)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [44]:
# get all the trades where fills is not []
fills = trades_df[trades_df.fills.map(len) > 0]

fills_list = list(fills["order"])

# loop through fills and convert each member of this data type to a row in a df
#  Order(permId=253828901, action='BUY', orderType='LMT', lmtPrice=17580.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder')

[Order(permId=253828896, action='BUY', orderType='LMT', lmtPrice=17603.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder'),
 Order(permId=253828901, action='BUY', orderType='LMT', lmtPrice=17580.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder'),
 Order(permId=253828900, action='SELL'